***Question 1:***

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests

In [2]:
postcode_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
indexname = postcode_data[postcode_data['Borough'] == 'Not assigned'].index
newdata = postcode_data.drop(indexname, inplace = False).reset_index(drop=True)
newdata.head()


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
newdata.shape

(103, 3)

***Question 2:***

In [4]:
latlng = pd.read_csv('Geospatial_Coordinates.csv')
newdata = pd.merge(newdata, latlng, how='inner', on='Postal Code')
newdata
#latlng

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


***Question 3:***

In [5]:
# get location data for Toronto
address = 'Toronto, ON, Canada'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [6]:
# create follium map object centered on toronto
torontomap = folium.Map(location=[latitude,longitude], zoomstart = 8)
# create Toronto data frame
torontodf = newdata[newdata['Borough'].str.contains('Toronto')].reset_index(drop=True)
#add neighborohood markers
for label,lat, lng in zip(torontodf['Neighbourhood'],
                          torontodf['Latitude'],
                          torontodf['Longitude']):
    folium.CircleMarker([lat,lng],
                        popup = label,
                        radius = 6,
                        fill = True,
                        color = 'red',
                        parse_html = True).add_to(torontomap)
    

    
torontomap

In [7]:
# Create client details for Foursqaure
CLIENT_ID = 'HPQHEBGSIPTCMJMDV4OSBP1N4FLKHXMTCONYZ54QNMK42NZE' # your Foursquare ID
CLIENT_SECRET = '4XHQ5ME1BZSMV13B34IXUWKGI12DDWP4SI5BYBOIINBL010Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HPQHEBGSIPTCMJMDV4OSBP1N4FLKHXMTCONYZ54QNMK42NZE
CLIENT_SECRET:4XHQ5ME1BZSMV13B34IXUWKGI12DDWP4SI5BYBOIINBL010Z


In [8]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    latitude, longitude,
    VERSION,
    radius,
    LIMIT)
results = requests.get(url).json()

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venue = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venue) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in nearby_venues.columns if col.startswith('venue.location.')] + ['venue.id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

nearby_venues.head(50)

<ipython-input-10-d95d8348bfc4>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venue) # flatten JSON


,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,Downtown Toronto,Neighborhood,43.653232,-79.385296,"[{'label': 'display', 'lat': 43.65323167517444...",113,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,5227bb01498e17bf485e6202
1,Nathan Phillips Square,Plaza,43.652270,-79.383516,"[{'label': 'display', 'lat': 43.65227047322295...",138,CA,Toronto,ON,Canada,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...",100 Queen St W,at Bay St,M5H 2N1,NaN,4ad4c05ef964a520a6f620e3
2,Japango,Sushi Restaurant,43.655268,-79.385165,"[{'label': 'display', 'lat': 43.65526771691681...",222,CA,Toronto,ON,Canada,"[122 Elizabeth St. (at Dundas St. W), Toronto ...",122 Elizabeth St.,at Dundas St. W,M5G 1P5,NaN,4ae7b27df964a52068ad21e3
3,Poke Guys,Poke Place,43.654895,-79.385052,"[{'label': 'display', 'lat': 43.65489527525682...",181,CA,Toronto,ON,Canada,"[112 Elizabeth St (at Dundas St W), Toronto ON...",112 Elizabeth St,at Dundas St W,M5G 1P5,NaN,57bcd3b7498e652a678d0378
4,Textile Museum of Canada,Art Museum,43.654396,-79.386500,"[{'label': 'display', 'lat': 43.65439630500274...",230,CA,Toronto,ON,Canada,[55 Centre Avenue (University Ave. and Dundas ...,55 Centre Avenue,University Ave. and Dundas St W.,M5G 2H5,NaN,4ad4c05ef964a520e2f620e3
5,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684,"[{'label': 'display', 'lat': 43.65554164147378...",237,CA,Toronto,ON,Canada,"[132 Dundas St W (btwn Bay & University), Toro...",132 Dundas St W,btwn Bay & University,M5G 1C3,NaN,4e2284b11fc7c0ef9857d143
6,LUSH,Cosmetics Shop,43.653557,-79.380400,"[{'label': 'display', 'lat': 43.653557, 'lng':...",284,CA,Toronto,ON,Canada,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","220 Yonge St, Unit B215-A",in Toronto Eaton Centre,M5B 2H1,NaN,4bd0b30d41b9ef3b8fa0fae5
7,CF Toronto Eaton Centre,Shopping Mall,43.654447,-79.380952,NaN,263,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",220 Yonge St,btwn Queen & Dundas,M5B 2H1,"Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
8,Indigo,Bookstore,43.653515,-79.380696,"[{'label': 'display', 'lat': 43.65351471121164...",260,CA,Toronto,ON,Canada,"[220 Yonge St, Toronto ON M5B 2H1, Canada]",220 Yonge St,NaN,M5B 2H1,Downtown Yonge,4b2a6eb8f964a52012a924e3
9,M Square Coffee Co,Coffee Shop,43.651218,-79.383555,"[{'label': 'display', 'lat': 43.65121797253777...",253,CA,Toronto,ON,Canada,"[C24 - 123 Queen St W, Toronto ON, Canada]",C24 - 123 Queen St W,NaN,NaN,NaN,54132b3b498ee9ca9332e189


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
toronto_venues = getNearbyVenues(names=torontodf['Neighbourhood'],
                        latitudes=torontodf['Latitude'],
                        longitudes=torontodf['Longitude'])
toronto_venues.groupby('Neighborhood').count()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
# Category frequency
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.016393,0.000000,0.000000,0.016393
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025316,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,...,0.012658,0.012658,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000


In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Cheese Shop,Restaurant,Beer Bar,Café,Concert Hall
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Restaurant,Burrito Place,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Comic Shop,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Plane,Rental Car Location,Harbor / Marina,Bar,Coffee Shop,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Portuguese Restaurant,Ramen Restaurant


In [15]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontodf

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head(20) # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Restaurant,Café,Theater,Bank,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Japanese Restaurant,Smoothie Shop,Beer Bar,Italian Restaurant,Diner,Burrito Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Italian Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Café,Cosmetics Shop,Movie Theater,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Hotel,Bakery,Gym,Restaurant,Department Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Park,Trail,Pub,Health Food Store,Wine Bar,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Cheese Shop,Restaurant,Beer Bar,Café,Concert Hall
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Portuguese Restaurant,Ramen Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Baby Store,Restaurant,Candy Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Nightclub
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Restaurant,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Salad Place,Hotel,Bakery
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Pet Store,Supermarket,Park,Music Venue,Pool,Middle Eastern Restaurant,Café,Brewery


In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [18]:
cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster0

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Breakfast Spot,Restaurant,Café,Theater,Bank,Chocolate Shop
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Japanese Restaurant,Smoothie Shop,Beer Bar,Italian Restaurant,Diner,Burrito Place
2,Downtown Toronto,0,Coffee Shop,Clothing Store,Italian Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Café,Cosmetics Shop,Movie Theater,Hotel
3,Downtown Toronto,0,Café,Coffee Shop,Cocktail Bar,American Restaurant,Gastropub,Hotel,Bakery,Gym,Restaurant,Department Store
5,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Farmers Market,Cheese Shop,Restaurant,Beer Bar,Café,Concert Hall
6,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Burger Joint,Salad Place,Poke Place,Portuguese Restaurant,Ramen Restaurant
7,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Restaurant,Candy Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Nightclub
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Clothing Store,Gym,Deli / Bodega,Thai Restaurant,Salad Place,Hotel,Bakery
9,West Toronto,0,Pharmacy,Bakery,Pet Store,Supermarket,Park,Music Venue,Pool,Middle Eastern Restaurant,Café,Brewery
10,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Italian Restaurant,Bank


In [19]:
from collections import Counter 
  
def most_frequent(toronto_merged): 
    occurence_count = Counter(toronto_merged) 
    return occurence_count.most_common(1)[0][0] 
    
List = [2, 1, 2, 2, 1, 3] 
print(most_frequent(toronto_merged['1st Most Common Venue'])) 

Coffee Shop


Not too suprising when you view the above data frame, however it appears downtown toronto has the most coffee shops. The map above suggests the other neighborhood clusters may have a unique element in the sense that its most common venue will likely differ, lets have a look.

In [21]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,1,Summer Camp,Restaurant,Wine Bar,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


This cluster shows a lot smaller of a sample size to make comparisons from but it does highlight a different venue that is the most common, this could be for a number of reasons. Looking back through the neighborhoods of this cluster it appears of them contains a park which highlights a distinctive feature compared to the other neighbors.

In [23]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Home Service,Garden,Wine Bar,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


This cluster demonstrates close similarities in common venues to cluster 3 in that many of the common venues are the same however the top 2 are only included in this cluster so far. It is appearing that it could be unique venues to the neighborhood that are defining the clusters at the moment, lets have a look at the next cluster

In [24]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Park,Trail,Pub,Health Food Store,Wine Bar,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store
33,Downtown Toronto,3,Park,Playground,Trail,Wine Bar,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


cluster 3 is also demonstrating unique venues not seen in the previous clusters in the 2nd most common venue column.

In [25]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,4,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Wine Bar,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 4 is showing a trail as a common category whi9ch is the same as the previous category however it also features a jewelry store which hasn't been seen in a cluster yet 

In [26]:
cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,5,Bus Line,Park,Swim School,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 5 also contains a unique value as its most common venue.

Overall it is very hard to say what is making the clusters unique but it appears they could be some relationship between the instance of a occurance of the venue on both ends of the spectrum.